In [9]:
import fmpy
import numpy as np
from fmpy.fmi2 import FMU2Slave
from fmpy.util import plot_result
import numpy as np


In [ ]:
filename = 'SolarTestModel.fmu'

start_values = {
    # variable                                       start   unit            description
    'TIn.TAmb':                                    (293.15, 'K'),          # Fixed ambient temperature for heat transfer
    'TIn.T_start':                                 (293.15, 'K'),          # Initial or guess value of output (= state)
    'TIn.tauHeaTra':                               (1200.0, 's'),          # Time constant for heat transfer, default 20 minutes
    'TOut.TAmb':                                   (293.15, 'K'),          # Fixed ambient temperature for heat transfer
    'TOut.T_start':                                (293.15, 'K'),          # Initial or guess value of output (= state)
    'TOut.tauHeaTra':                              (1200.0, 's'),          # Time constant for heat transfer, default 20 minutes
    'exp.T_start':                                 (293.15, 'K'),          # Start value of temperature
    'exp.V_start':                                    (0.1, 'm3'),         # Volume of liquid stored in the vessel at the start of the simulation
    'exp.X_start[1]':                                 (1.0, 'kg/kg'),      # Start value of mass fractions m_i/m
    'exp.p_start':                                    (3e5, 'Pa'),         # Start value of pressure
    'pum.T_start':                                 (293.15, 'K'),          # Start value of temperature
    'pum.X_start[1]':                                 (1.0, 'kg/kg'),      # Start value of mass fractions m_i/m
    'pum.dp_start':                                   (0, 'Pa'),         # Initial value of pressure raise
    'pum.m_flow_small':              (4.000000000000001e-6, 'kg/s'),       # Small mass flow rate for regularization of zero flow
    'pum.p_start':                                    (3e5, 'Pa'),         # Start value of pressure
    'pum.riseTime':                                  (30.0, 's'),          # Rise time of the filter (time to reach 99.6 % of the speed)
    'pum.tau':                                        (1.0, 's'),          # Time constant of fluid volume for nominal flow, used if energy or mass balance is dynamic
    'rooT.T':                                      (293.15, 'K'),          # Fixed temperature at port
    'solCol.T_start':                              (293.15, 'K'),          # Start value of temperature
    'solCol.X_start[1]':                              (1.0, 'kg/kg'),      # Start value of mass fractions m_i/m
    'solCol.azi':                                     (0.0, 'rad'),        # Surface azimuth (0 for south-facing; -90 degree for east-facing; +90 degree for west facing
    'solCol.p_start':                                 (3e5, 'Pa'),         # Start value of pressure
    'solCol.per.A':                                 (2.003, 'm2'),         # Area
    'solCol.per.B0':                                -0.137,                # 1st incident angle modifier coefficient
    'solCol.per.B1':                                0.0166,                # 2nd incident angle modifier coefficient
    'solCol.per.C1':                                   0.0,                # Heat loss coefficient from EN12975 ratings data
    'solCol.per.C2':                                   0.0,                # Temperature dependence of heat loss from EN12975 ratings data
    'solCol.per.G_nominal':                        (1000.0, 'W/m2'),       # Nominal solar irradiance specified in ratings data
    'solCol.per.IAMDiff':                              0.0,                # Incidence angle modifier from EN12975 ratings data
    'solCol.per.V':                                (0.0038, 'm3'),         # Fluid volume
    'solCol.per.dT_nominal':                         (10.0, 'K'),        
    'solCol.per.dp_nominal':                        (93.89, 'Pa'),         # Pressure drop during test conditions
    'solCol.per.mDry':                               (42.0, 'kg'),         # Dry weight
    'solCol.per.mperA_flow_nominal':               (0.0194, 'kg/(s.m2)'),  # Mass flow rate per unit area of collector
    'solCol.per.slope':                            (-5.103, 'W/(m2.K)'),   # Slope from rating data
    'solCol.per.y_intercept':                        0.775,                # Y intercept (Maximum efficiency)
    'solCol.rho':                                     (0.2, '1'),          # Ground reflectance
    'solCol.shaCoe':                                   0.0,                # Shading coefficient. 0.0: no shading, 1.0: full shading
    'solCol.til':                                     (0.0, 'rad'),        # Surface tilt (0 for horizontally mounted collector)
    'solCol.totalArea':                               (0.0, 'm2'),         # Total area of panels in the simulation
    'tan.T_start':                                 (293.15, 'K'),          # Start value of temperature
    'tan.VTan':                                       (1.5, 'm3'),         # Tank volume
    'tan.X_start[1]':                                 (1.0, 'kg/kg'),      # Start value of mass fractions m_i/m
    'tan.dIns':                                      (0.07, 'm'),          # Thickness of insulation
    'tan.hTan':                                       (1.8, 'm'),          # Height of tank (without insulation)
    'tan.kIns':                                      (0.04, 'W/(m.K)'),    # Specific heat conductivity of insulation
    'tan.p_start':                                    (3e5, 'Pa'),         # Start value of pressure
    'tan.tau':                                        (1.0, 's'),          # Time constant for mixing
    'weaDat.HInfHor':                                 (0.0, 'W/m2'),       # Infrared horizontal radiation (used if HInfHorSou=Parameter)
    'weaDat.TBlaSky':                              (273.15, 'K'),          # Black-body sky temperature (used if TBlaSkySou=Parameter)
    'weaDat.TDewPoi':                              (283.15, 'K'),          # Dew point temperature (used if TDewPoi=Parameter)
    'weaDat.TDryBul':                              (293.15, 'K'),          # Dry bulb temperature (used if TDryBul=Parameter)
    'weaDat.ceiHei':                                  (2e4, 'm'),          # Ceiling height (used if ceiHei=Parameter)
    'weaDat.opaSkyCov':                               (0.5, '1'),          # Opaque sky cover (used if opaSkyCov=Parameter). Use 0 <= opaSkyCov <= 1
    'weaDat.pAtm':                               (101325.0, 'Pa'),         # Atmospheric pressure (used if pAtmSou=Parameter)
    'weaDat.relHum':                                  (0.5, '1'),          # Relative humidity (used if relHum=Parameter)
    'weaDat.totSkyCov':                               (0.5, '1'),          # Total sky cover (used if totSkyCov=Parameter). Use 0 <= totSkyCov <= 1
    'weaDat.winDir':                                  (1.0, 'rad'),        # Wind direction (used if winDir=Parameter)
    'weaDat.winSpe':                                  (1.0, 'm/s'),        # Wind speed (used if winSpe=Parameter)
    'solCol.nColType':                                   1,                # Selection of area specification format
    'solCol.nPanels':                                    5,                # Desired number of panels in the simulation
    'solCol.per.ATyp':                                   1,                # Gross or aperture area
    'solCol.sysConfig':                                  2,                # Selection of system configuration
}

output = [
    'TIn.T',            # Temperature of the passing fluid
    'TOut.T',           # Temperature of the passing fluid
    'TTan.T',           # Temperature of buffer
    'weaDat.weaBus.HGloHor', #irradiance
    'flowSensor.m_flow', #flow through circuit
    'pum.dp_actual', #pressure difference pump 
]

input = ['pump_dp_in']

result = fmpy.simulate_fmu(filename, start_values=start_values, output=output, stop_time=24*60*60, solver = 'Euler', step_size = 1)

fmpy.plot_result(result)


In [ ]:
#control function that starts the pump when irradiance is above threshold
def get_control(results, vr_inputs):

    irradiance = results[-3]

    return (vr_inputs, [0.00 if irradiance <=100 else 1e3])

# define the simulation parameters
start_time = 0
stop_time = start_time + 3*24*60*60
step_size = 1

# read the model description
model_description = fmpy.read_model_description(filename)


# get the value references for the variables we want to get/set
vrs = {variable.name:variable.valueReference for variable in model_description.modelVariables}
vr_inputs   = [vrs[var] for var in input] #control
vr_outputs = [vrs[var] for var in output] #sensors

# extract the FMU
fmu = FMU2Slave(guid=model_description.guid,
                unzipDirectory=fmpy.extract(filename),
                modelIdentifier=model_description.coSimulation.modelIdentifier,
                instanceName='instance1')

# initialize FMU
fmu.instantiate()
fmu.setupExperiment(startTime=start_time)
fmu.enterInitializationMode()
fmu.exitInitializationMode()

time = start_time

rows = []  # list to record the results

control_values = (vr_inputs, [0.00]) #initial control values 

# simulation loop
while time < stop_time:

    # NOTE: the FMU.get*() and FMU.set*() functions take lists of value references as arguments and return lists of values

    # set the input
    fmu.setReal(*control_values)

    # perform one step
    fmu.doStep(currentCommunicationPoint=time, communicationStepSize=step_size)

    # advance the time
    time += step_size

    # get the values for 'inputs' and 'outputs'
    results = fmu.getReal(vr_inputs + vr_outputs)

    control_values = get_control(results, vr_inputs)

    # append the results
    rows.append((time, *results))


# clean up
fmu.terminate()
fmu.freeInstance()

# convert the results to a structured NumPy array
result = np.array(rows, dtype=np.dtype([(var, np.float64) for var in ['time'] + input + output]))

# plot the results
plot_result(result)